# Assignment Solution Notebook


Problem Solving Approach:

1.Understanding the data and different classes to answer of questions into.
2.Will try to take 75% questions to train and rest 25% to test
3.Output class will be a one hot vector
4.Input will be question and its options,  Will use word2vec or universal sentence encoder to compute embedding of a sentence to feed as an attribute of a question.
5.Starting with a simple 2 layer neural network to see how it performs.
6.Tune the network, used relu, apropiate lr and epochs.
7.Plan to use word2vec and for whole sentence, averaging them out to get one vector for the complete sentence.




In [19]:
#getting the data, cleaning, preprocssing and structuring 
import json
with open('train.csv') as f:
    content = f.readlines()

#format: id,answerA,answerB,answerC,answerD,correctAnswer,question
data = []
for i in xrange(1,len(content)):
    
    flag = 0
    #Removing comma between quotes, i.e. if there is a comma in any of the options
    newline = ''
    for j  in range(len(content[i])):
        if(content[i][j]) == '"':
            flag = 1-flag
            
        if flag == 1 and content[i][j] == ',':
            continue
        else:
            newline += content[i][j]
    
    line = newline.split(',')
    d = {}
    d['id'] = line[0]
    d['question'] = line[6][:-1]
    d['A'] = line[1]
    d['B'] = line[2]
    d['C'] = line[3]
    d['D'] = line[4]
    d['answer'] = line[ord(line[5]) - ord('A') + 1]
    d['output'] = line[5]
    data.append(d)
    #print(line)
#     print(d)
#     break

In [36]:
import json
with open('test.csv') as f:
    content = f.readlines()

#format: id,answerA,answerB,answerC,answerD,question
testing_data = []
for i in xrange(1,len(content)):
    
    flag = 0
    #Removing comma between quotes, i.e. if there is a comma in any of the options
    newline = ''
    for j  in range(len(content[i])):
        if(content[i][j]) == '"':
            flag = 1-flag
            
        if flag == 1 and content[i][j] == ',':
            continue
        else:
            newline += content[i][j]
    
    line = newline.split(',')
    d = {}
    d['id'] = line[0]
    d['question'] = line[5][:-1]
    d['A'] = line[1]
    d['B'] = line[2]
    d['C'] = line[3]
    d['D'] = line[4]
    testing_data.append(d)

In [21]:
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np

#Resusable functions keeping the problem and preprocessing of text in mind

import nltk
nltk.download('punkt')

import gensim

# Load Google's pre-trained Word2Vec model.
word_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

def preprocess(sen):
    if sen == "":
        return ""
    
    if type(sen) is list:
        s = ""
        for word in sen:
            s += word + ' '
        sen = s[:-1]
    sen = sen.replace("&nbsp;", ' ')
    sen = sen.replace("&amp;", ' ')
    sen = sen.replace('.', ' ')
    sen = sen.replace(',', ' ')
    sen = sen.lower()
    sen = re.sub(r'\d+', '', sen)
    
    #sen = sen.translate(string.maketrans('','', string.punctuation))
    exclude = set(string.punctuation)
    for c in exclude:
        sen = sen.replace(c, ' ')
    sen = ''.join(ch for ch in sen if ch not in exclude)
    sen = sen.strip()
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(sen)
    sen = [i for i in tokens if not i in stop_words]
    return sen

def embed(sen):
    
    sen = preprocess(sen)
    #print(sen)
    emm = np.ones(300)/100
    em = np.zeros(300)
    count = 0
    words_not_found = 0
    for w in sen:
        try:
            em += word_model[w]
            count+=1
        except KeyError:
            #print(w, "Not found")
            words_not_found += 1
            continue
    
    #print(em/count)
    if sen == "" or count == 0:
        return emm
    return em/count

[nltk_data] Downloading package punkt to /Users/anshul/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [29]:
#Trying out neural network approach
#Creating respective datasets


import numpy as np
x_train = []
x_test = []
y_train = []
y_test = []

for d in data:
    
    t = []
    t.append(embed(d['question']))
    t.append(embed(d['A']))
    t.append(embed(d['B']))
    t.append(embed(d['C']))
    t.append(embed(d['D']))
    x_train.append(t)
    
    output = np.zeros(4)
    output[ord(d['output']) - ord('A')] = 1
    y_train.append(output)
    
x_train = np.array(x_train)
y_train = np.array(y_train)

In [37]:
for d in testing_data:
    t = []
    t.append(embed(d['question']))
    t.append(embed(d['A']))
    t.append(embed(d['B']))
    t.append(embed(d['C']))
    t.append(embed(d['D']))
    x_test.append(t)
    
#     output = np.zeros(4)
#     output[ord(d['output']) - ord('A')] = 1
#     y_train.append(output)

x_test = np.array(x_test)
print(x_test.shape)
x_test = x_test.reshape(len(x_test), 1500)
print(x_test.shape)

(516, 5, 300)
(516, 1500)


In [32]:
print(x_train.shape, y_train.shape)
x_train = x_train.reshape(len(x_train, 1500)
print(x_train.shape, y_train.shape)

((4610, 5, 300), (4610, 4))
((4610, 1500), (4610, 4))


In [42]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import optimizers
from keras.regularizers import l2

model = Sequential([
    Dense(1240, input_shape=(1500,),  W_regularizer=l2(0.01)),
    Activation('relu'),
    Dense(124, W_regularizer=l2(0.01)),
    Activation('relu'),
    Dense(4, W_regularizer=l2(0.01)),
    Activation('softmax')
])
#opt = optimizers.sgd(lr=0.0001, decay=1e-6)
opt = optimizers.Adam(lr=0.0001, decay=1e-6)

#sgd = optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.4, nesterov=False)
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

/anaconda3/envs/deeplearningproject/lib/python2.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1240, kernel_regularizer=<keras.reg..., input_shape=(1500,))`
  import sys
/anaconda3/envs/deeplearningproject/lib/python2.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(124, kernel_regularizer=<keras.reg...)`
  if __name__ == '__main__':
/anaconda3/envs/deeplearningproject/lib/python2.7/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(4, kernel_regularizer=<keras.reg...)`
  # This is added back by InteractiveShellApp.init_path()


# Training of model on train.csv, gives me around 54% accuracy

In [43]:
model.fit(x_train, y_train, epochs=100, batch_size=64,verbose=1)

Epoch 1/100
4610/4610 [==============================] - 6s - loss: 14.7307 - acc: 0.2579     
Epoch 2/100
4610/4610 [==============================] - 4s - loss: 10.4062 - acc: 0.3273     
Epoch 3/100
4610/4610 [==============================] - 3s - loss: 7.4223 - acc: 0.3766     
Epoch 4/100
4610/4610 [==============================] - 2s - loss: 5.3857 - acc: 0.4130     
Epoch 5/100
4610/4610 [==============================] - 2s - loss: 4.0172 - acc: 0.4041     
Epoch 6/100
4610/4610 [==============================] - 3s - loss: 3.1111 - acc: 0.4215     
Epoch 7/100
4610/4610 [==============================] - 3s - loss: 2.5207 - acc: 0.4065     
Epoch 8/100
4610/4610 [==============================] - 3s - loss: 2.1371 - acc: 0.4100     
Epoch 9/100
4610/4610 [==============================] - 3s - loss: 1.8897 - acc: 0.4321     - ETA: 0s - loss: 1.9042
Epoch 10/100
4610/4610 [==============================] - 3s - loss: 1.7305 - acc: 0.4265     
Epoch 11/100
4610/4610 [=========

4610/4610 [==============================] - 4s - loss: 1.3832 - acc: 0.5236     
Epoch 86/100
4610/4610 [==============================] - 3s - loss: 1.3802 - acc: 0.5239     
Epoch 87/100
4610/4610 [==============================] - 3s - loss: 1.3811 - acc: 0.5239     
Epoch 88/100
4610/4610 [==============================] - 3s - loss: 1.3824 - acc: 0.5161     
Epoch 89/100
4610/4610 [==============================] - 3s - loss: 1.3787 - acc: 0.5332     
Epoch 90/100
4610/4610 [==============================] - 2s - loss: 1.3786 - acc: 0.5302     
Epoch 91/100
4610/4610 [==============================] - 2s - loss: 1.3812 - acc: 0.5328     
Epoch 92/100
4610/4610 [==============================] - 2s - loss: 1.3778 - acc: 0.5406     
Epoch 93/100
4610/4610 [==============================] - 3s - loss: 1.3893 - acc: 0.5156     
Epoch 94/100
4610/4610 [==============================] - 4s - loss: 1.3780 - acc: 0.5347     
Epoch 95/100
4610/4610 [==============================] - 4s - 

In [44]:
Y_test = model.predict(x_test)

In [56]:
##Creating the submission file
f = open('submission', 'w')
f.write('id,correctAnswer\n')
i = 0
for d in testing_data:
    
    ans = str(d['id']) + ',' + chr(65 + np.argmax(Y_test[i])) + '\n'
    f.write(ans)
    i += 1
    
f.close()

# Trying the split just to get the idea of accuracy of my model on testing data
since I have not been given with the correct answers of question in 'test.csv'

In [60]:
x_train = []
x_test = []
y_train = []
y_test = []

thresh = .75*len(data)
i = 0

for d in data:
    
    t = []
    t.append(embed(d['question']))
    t.append(embed(d['A']))
    t.append(embed(d['B']))
    t.append(embed(d['C']))
    t.append(embed(d['D']))
    output = np.zeros(4)
    output[ord(d['output']) - ord('A')] = 1
    
    if i < thresh:
        x_train.append(t)
        y_train.append(output)
    else:
        x_test.append(t)
        y_test.append(output)
        
    i += 1
    
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)
x_test = x_test.reshape(len(x_test), 1500)
x_train = x_train.reshape(len(x_train), 1500)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

((3458, 1500), (3458, 4))
((1152, 1500), (1152, 4))


In [68]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import optimizers
from keras.regularizers import l2

split_model = Sequential([
    Dense(1240, input_shape=(1500,),  W_regularizer=l2(0.01)),
    Activation('relu'),
    Dense(124, W_regularizer=l2(0.01)),
    Activation('relu'),
    Dense(4, W_regularizer=l2(0.01)),
    Activation('softmax')
])
#opt = optimizers.sgd(lr=0.0001, decay=1e-6)
opt = optimizers.Adam(lr=0.0001, decay=1e-6)

#sgd = optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.4, nesterov=False)
split_model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

/anaconda3/envs/deeplearningproject/lib/python2.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1240, kernel_regularizer=<keras.reg..., input_shape=(1500,))`
  import sys
/anaconda3/envs/deeplearningproject/lib/python2.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(124, kernel_regularizer=<keras.reg...)`
  if __name__ == '__main__':
/anaconda3/envs/deeplearningproject/lib/python2.7/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(4, kernel_regularizer=<keras.reg...)`
  # This is added back by InteractiveShellApp.init_path()


In [69]:
split_model.fit(x_train, y_train, epochs=100, batch_size=64,verbose=1)

Epoch 1/100
3458/3458 [==============================] - 3s - loss: 15.3897 - acc: 0.2585     
Epoch 2/100
3458/3458 [==============================] - 3s - loss: 11.9226 - acc: 0.3548     
Epoch 3/100
3458/3458 [==============================] - 3s - loss: 9.2944 - acc: 0.4158     
Epoch 4/100
3458/3458 [==============================] - 3s - loss: 7.3059 - acc: 0.4436     
Epoch 5/100
3458/3458 [==============================] - 2s - loss: 5.8090 - acc: 0.4584     
Epoch 6/100
3458/3458 [==============================] - 3s - loss: 4.6891 - acc: 0.4639     
Epoch 7/100
3458/3458 [==============================] - 2s - loss: 3.8509 - acc: 0.4795     
Epoch 8/100
3458/3458 [==============================] - 2s - loss: 3.2289 - acc: 0.4705     
Epoch 9/100
3458/3458 [==============================] - 2s - loss: 2.7664 - acc: 0.4844     
Epoch 10/100
3458/3458 [==============================] - 3s - loss: 2.4286 - acc: 0.4890     
Epoch 11/100
3458/3458 [==============================] -

3458/3458 [==============================] - 3s - loss: 1.3011 - acc: 0.7126     
Epoch 87/100
3458/3458 [==============================] - 3s - loss: 1.2908 - acc: 0.7253     
Epoch 88/100
3458/3458 [==============================] - 3s - loss: 1.2899 - acc: 0.7319     
Epoch 89/100
3458/3458 [==============================] - 3s - loss: 1.2932 - acc: 0.7290     
Epoch 90/100
3458/3458 [==============================] - 2s - loss: 1.2872 - acc: 0.7290     
Epoch 91/100
3458/3458 [==============================] - 2s - loss: 1.2839 - acc: 0.7392     
Epoch 92/100
3458/3458 [==============================] - 2s - loss: 1.2937 - acc: 0.7209     
Epoch 93/100
3458/3458 [==============================] - 2s - loss: 1.2846 - acc: 0.7464     
Epoch 94/100
3458/3458 [==============================] - 2s - loss: 1.2802 - acc: 0.7415     
Epoch 95/100
3458/3458 [==============================] - 2s - loss: 1.2825 - acc: 0.7392     
Epoch 96/100
3458/3458 [==============================] - 2s - 

In [70]:
score = split_model.evaluate(x_test, y_test, batch_size=64)
print("Model metrics: ",split_model.metrics_names)
print("TEST ACCURACY: ", score[1])

 896/1152 [======================>.......] - ETA: 0s('Model metrics: ', ['loss', 'acc'])
('TEST ACCURACY: ', 0.2855902777777778)
